# Concept of AgentMemory package

## Notebook Initialization

In [1]:
import sys

sys.path.append("../src")

In [2]:
from agentmemory import AgentMemory
from agentmemory.connection import AgentMemoryConnection
from agentmemory.connection.longterm import MongoDBConnection
from agentmemory.connection.shortterm import RedisConnection

con = AgentMemoryConnection(
    longterm_con=MongoDBConnection(
        mongo_uri="mongodb://localhost:27017",
        database="agentmemory-dev"
    ),
    shortterm_con=RedisConnection(
        host="localhost"
    )
)

memory = AgentMemory("typecode-agent", con=con)

memory.conversations

In [3]:
def delete_all_conversations(cascade: bool = True) -> None:
    for conversation in memory.conversations.list():
        memory.conversations.delete(conversation.conversation_id, cascade=cascade)

In [4]:
delete_all_conversations()

CLEAR KEYS: [b'type:LIST;col:conversations;'] 3


## Datamodel

### Conversations

In [5]:
from agentmemory.schema.conversations import Conversation
from uuid import uuid4

conversation = Conversation(
    title="New Conversation 1" + str(uuid4()),
)

conversation_2 = Conversation(
    title="New Conversation 2" + str(uuid4()),
)

res = memory.conversations.create(conversation)
res_2 = memory.conversations.create(conversation_2)
res

Conversation(title='New Conversation 10a264292-8b6c-4980-8bf4-c5a845a9aa22', _id='687a02edef4dc38c5e919c5b', conversation_id='625388c4f4a74a348ca32b03cb716721', data={}, created_at='2025-07-18T08:16:45.699513+00:00', updated_at='2025-07-18T08:16:45.699530+00:00')

In [6]:
res.data

{}

In [6]:
l = [d for d in memory.conversations.list()]
l

[Conversation(title='New Conversation 1832ec493-38f4-4677-848f-05e4e87e8ea8', _id='6879458acdf5f3fc801b9d38', conversation_id='38af280040d54bf7a79da31529255a86', data='{}', created_at='2025-07-17T18:48:42.577479+00:00', updated_at='2025-07-17T18:48:42.577491+00:00'),
 Conversation(title='New Conversation 297469b01-f0ee-4ca1-9c76-25b94ceb2bc6', _id='6879458acdf5f3fc801b9d39', conversation_id='be5c8a17f8254f3f8b38f40c1c035115', data='{}', created_at='2025-07-17T18:48:42.577545+00:00', updated_at='2025-07-17T18:48:42.577549+00:00')]

In [7]:
conversation.title = "Updated Conversation Title"

updated_conversation = memory.conversations.update(res)
updated_conversation

CLEAR KEYS: [b'type:LIST;col:conversations;'] 3


Conversation(title='Updated Conversation Title', _id='6879458acdf5f3fc801b9d38', conversation_id='38af280040d54bf7a79da31529255a86', data={}, created_at='2025-07-17T18:48:42.577479+00:00', updated_at='2025-07-17T18:49:02.704682+00:00')

In [ ]:
memory.conversations.get(updated_conversation.conversation_id)

Conversation(title='Updated Conversation Title', _id=ObjectId('6874ecf36a0f76356d561c80'), conversation_id='175c08ce31424d44afffb45d89a8516b', data={}, created_at='2025-07-14T11:41:39.158050+00:00', updated_at='2025-07-14T11:41:43.392486+00:00')

In [ ]:
l = [d for d in memory.conversations.list()]
l

In [ ]:
r = memory.conversations.delete(updated_conversation.conversation_id, cascade=True)
r

In [ ]:
conversation_1 = next(memory.conversations.list())
conversation_1

Conversation(title='New Conversation 287d44d7e-74c8-4202-a5bc-873022007750', _id=ObjectId('6874c395608cd154cce9fc81'), conversation_id='3eba2baa2d034db888a85f8d1d393e6e', data={}, created_at='2025-07-14T08:45:09.370220+00:00', updated_at='2025-07-14T08:45:09.370236+00:00')

In [ ]:
r = memory.conversations.delete(conversation_1.conversation_id, cascade=True)
r

### ConversationItems

In [ ]:
from agentmemory.schema.conversations import ConversationItem

item = ConversationItem(
    conversation_id=conversation_1.conversation_id,
    # conversation_id="b000184d255b46b694d475b57db67881",
    content="New item content",
    role="user",
    data={"source": "test_source"}
)
item_created = memory.conversation_items.create(item)
item_created

ConversationItem(conversation_id='3eba2baa2d034db888a85f8d1d393e6e', role='user', content='New item content', _id='6874ed436a0f76356d561c83', item_id='33dde36760ad407e865d7462452b4745', data={'source': 'test_source'}, created_at='2025-07-14T11:42:59.892681+00:00', updated_at='2025-07-14T11:42:59.892694+00:00')

In [ ]:
[d for d in memory.conversation_items.list()]

[ConversationItem(conversation_id='b000184d255b46b694d475b57db67881', role='user', content='New item content', _id=ObjectId('6874dc6f9904626b8cb1d8fe'), item_id='9b3da64095304f5aa69105502eda501e', data={'source': 'test_source'}, created_at='2025-07-14T10:31:11.995088+00:00', updated_at='2025-07-14T10:31:11.995094+00:00'),
 ConversationItem(conversation_id='b000184d255b46b694d475b57db67881', role='user', content='New item content', _id=ObjectId('6874dc709904626b8cb1d8ff'), item_id='38f9b58285fa46879e1622a5a342c608', data={'source': 'test_source'}, created_at='2025-07-14T10:31:12.589647+00:00', updated_at='2025-07-14T10:31:12.589656+00:00'),
 ConversationItem(conversation_id='b000184d255b46b694d475b57db67881', role='user', content='New item content', _id=ObjectId('6874dc709904626b8cb1d900'), item_id='2fc2007b36d34655932410a590ffede0', data={'source': 'test_source'}, created_at='2025-07-14T10:31:12.956162+00:00', updated_at='2025-07-14T10:31:12.956171+00:00'),
 ConversationItem(conversati

In [ ]:
[i for i in memory.conversation_items.list_by_conversation_id(conversation_1.conversation_id)]

[ConversationItem(conversation_id='3eba2baa2d034db888a85f8d1d393e6e', role='user', content='New item content', _id=ObjectId('6874ed436a0f76356d561c83'), item_id='33dde36760ad407e865d7462452b4745', data={'source': 'test_source'}, created_at='2025-07-14T11:42:59.892681+00:00', updated_at='2025-07-14T11:42:59.892694+00:00')]

In [ ]:
[d for d in memory.conversation_items.list_until_id_found(conversation_1.conversation_id, "f36e0b5e303643f5bfec415ee9743abf")]

[ConversationItem(conversation_id='3eba2baa2d034db888a85f8d1d393e6e', role='user', content='Updated item content', _id=ObjectId('6874c3c9608cd154cce9fc82'), item_id='6114519c58ab44be957a2ed245bfb012', data={'source': 'test_source'}, created_at='2025-07-14T08:46:01.952297+00:00', updated_at='2025-07-14T08:47:31.082819+00:00'),
 ConversationItem(conversation_id='3eba2baa2d034db888a85f8d1d393e6e', role='user', content='New item content', _id=ObjectId('6874db8fb0e1bda8d1eabf03'), item_id='aae06f340f3f4b8a98f9608262e3707d', data={'source': 'test_source'}, created_at='2025-07-14T10:27:27.579065+00:00', updated_at='2025-07-14T10:27:27.579075+00:00'),
 ConversationItem(conversation_id='3eba2baa2d034db888a85f8d1d393e6e', role='user', content='New item content', _id=ObjectId('6874dc049904626b8cb1d8f9'), item_id='c9f9402ccc3d4f5491e5b7c235b63fef', data={'source': 'test_source'}, created_at='2025-07-14T10:29:24.174138+00:00', updated_at='2025-07-14T10:29:24.174147+00:00'),
 ConversationItem(conver

In [ ]:
memory.conversation_items.get(conversation_1.conversation_id, item_created.item_id)

ConversationItem(conversation_id='3eba2baa2d034db888a85f8d1d393e6e', role='user', content='New item content', _id=ObjectId('6874ed436a0f76356d561c83'), item_id='33dde36760ad407e865d7462452b4745', data={'source': 'test_source'}, created_at='2025-07-14T11:42:59.892681+00:00', updated_at='2025-07-14T11:42:59.892694+00:00')

In [ ]:
item_created.content = "Updated item content"
item_updated = memory.conversation_items.update(item_created)
item_updated

ConversationItem(conversation_id='3eba2baa2d034db888a85f8d1d393e6e', role='user', content='Updated item content', _id='6874ed436a0f76356d561c83', item_id='33dde36760ad407e865d7462452b4745', data={'source': 'test_source'}, created_at='2025-07-14T11:42:59.892681+00:00', updated_at='2025-07-14T11:43:29.625947+00:00')

In [ ]:
[i for i in memory.conversation_items.list_by_conversation_id(conversation_1.conversation_id)]

[]

In [ ]:
memory.conversation_items.delete(conversation_1.conversation_id, "33dde36760ad407e865d7462452b4745")

### Personas

In [ ]:
from agentmemory.schema.personas import Persona

persona = Persona(
    name="Test Persona",
    role="role",
    background="background",
    goals="This is a test persona"
)
persona_created = memory.personas.create(persona)  # TODO
persona_created

Persona(name='Test Persona', role='role', goals='This is a test persona', background='background', _id='6874ed8c6a0f76356d561c84', persona_id='90241714ab544d12bbacf546a66187dd', embedding=None, created_at='2025-07-14T11:44:12.443383+00:00', updated_at='2025-07-14T11:44:12.443393+00:00')

In [ ]:
memory.personas.get(persona_created.persona_id)

Persona(name='Test Persona', role='role', goals='This is a test persona', background='background', _id=ObjectId('6874ed8c6a0f76356d561c84'), persona_id='90241714ab544d12bbacf546a66187dd', embedding=None, created_at='2025-07-14T11:44:12.443383+00:00', updated_at='2025-07-14T11:44:12.443393+00:00')

In [ ]:
[p for p in memory.personas.list()]

[Persona(name='Updated Persona Name', role='role', goals='This is a test persona', background='background', _id=ObjectId('6874c535608cd154cce9fc84'), persona_id='caf483721e4a423b8cb4a9f8d664d90d', embedding=None, created_at='2025-07-14T08:52:05.201904+00:00', updated_at='2025-07-14T08:53:29.047120+00:00'),
 Persona(name='Test Persona', role='role', goals='This is a test persona', background='background', _id=ObjectId('6874ed8c6a0f76356d561c84'), persona_id='90241714ab544d12bbacf546a66187dd', embedding=None, created_at='2025-07-14T11:44:12.443383+00:00', updated_at='2025-07-14T11:44:12.443393+00:00')]

In [ ]:
[p for p in memory.personas.list()]

[Persona(name='Updated Persona Name', role='role', goals='This is a test persona', background='background', _id=ObjectId('6874c535608cd154cce9fc84'), persona_id='caf483721e4a423b8cb4a9f8d664d90d', embedding=None, created_at='2025-07-14T08:52:05.201904+00:00', updated_at='2025-07-14T08:53:29.047120+00:00'),
 Persona(name='Test Persona', role='role', goals='This is a test persona', background='background', _id=ObjectId('6874ed8c6a0f76356d561c84'), persona_id='90241714ab544d12bbacf546a66187dd', embedding=None, created_at='2025-07-14T11:44:12.443383+00:00', updated_at='2025-07-14T11:44:12.443393+00:00')]

In [ ]:
persona_created.name = "Updated Persona Name"
updated_persona = memory.personas.update(persona_created)
updated_persona

In [ ]:
memory.personas.get(persona_created.persona_id)

In [ ]:
memory.personas.delete(persona_created.persona_id)

### Workflows

In [ ]:
from agentmemory.schema.workflows import Workflow, WorkflowStatus
from uuid import uuid4

workflow = Workflow(
    conversation_item_id="blub-1",
    user_query="user query",
    status=WorkflowStatus.RUNNING,
    data={"source": "test_source"}
)

workflow_2 = Workflow(
    conversation_item_id="blub-1",
    user_query="user query 2",
    status=WorkflowStatus.ERROR,
    data={"source": "test_source"}
)

res = memory.workflows.create(workflow)
res_2 = memory.workflows.create(workflow_2)
res

Workflow(conversation_item_id='blub-1', user_query='user query', status=<WorkflowStatus.RUNNING: 'running'>, _id='6874edb36a0f76356d561c85', workflow_id='b4ab782ce1f04e8995e8061bd5102930', data={'source': 'test_source'}, created_at='2025-07-14T11:44:51.123921+00:00', updated_at='2025-07-14T11:44:51.123935+00:00')

In [ ]:
[w for w in memory.workflows.list()]

[Workflow(conversation_item_id='blub-1', user_query='user query 2', status='error', _id=ObjectId('6874c8627c0672cc31a5d0b7'), workflow_id='8f6c73497b034d51b232f7da17f4ea8b', data={'source': 'test_source'}, created_at='2025-07-14T09:05:38.731584+00:00', updated_at='2025-07-14T09:05:38.731588+00:00'),
 Workflow(conversation_item_id='blub-1', user_query='user query 2', status='error', _id=ObjectId('6874c8687c0672cc31a5d0b9'), workflow_id='5a42d02f5c53485796e32b0951c4c719', data={'source': 'test_source'}, created_at='2025-07-14T09:05:44.304595+00:00', updated_at='2025-07-14T09:05:44.304606+00:00')]

In [ ]:
memory.workflows.get(res.workflow_id)

Workflow(conversation_item_id='blub-1', user_query='user query', status='running', _id=ObjectId('6874edb36a0f76356d561c85'), workflow_id='b4ab782ce1f04e8995e8061bd5102930', data={'source': 'test_source'}, created_at='2025-07-14T11:44:51.123921+00:00', updated_at='2025-07-14T11:44:51.123935+00:00')

In [ ]:
res.status = WorkflowStatus.SUCCESS
res_updated = memory.workflows.update(res)
res_updated

Workflow(conversation_item_id='blub-1', user_query='user query', status=<WorkflowStatus.SUCCESS: 'success'>, _id='6874edb36a0f76356d561c85', workflow_id='b4ab782ce1f04e8995e8061bd5102930', data={'source': 'test_source'}, created_at='2025-07-14T11:44:51.123921+00:00', updated_at='2025-07-14T11:44:55.722165+00:00')

In [ ]:
[w for w in memory.workflows.list_by_conversation_id("blub-1")]

[Workflow(conversation_item_id='blub-1', user_query='user query 2', status='error', _id=ObjectId('6874c8627c0672cc31a5d0b7'), workflow_id='8f6c73497b034d51b232f7da17f4ea8b', data={'source': 'test_source'}, created_at='2025-07-14T09:05:38.731584+00:00', updated_at='2025-07-14T09:05:38.731588+00:00'),
 Workflow(conversation_item_id='blub-1', user_query='user query 2', status='error', _id=ObjectId('6874c8687c0672cc31a5d0b9'), workflow_id='5a42d02f5c53485796e32b0951c4c719', data={'source': 'test_source'}, created_at='2025-07-14T09:05:44.304595+00:00', updated_at='2025-07-14T09:05:44.304606+00:00'),
 Workflow(conversation_item_id='blub-1', user_query='user query 2', status='error', _id=ObjectId('6874edb36a0f76356d561c86'), workflow_id='21555e8acb01485191be678180fecb42', data={'source': 'test_source'}, created_at='2025-07-14T11:44:51.123991+00:00', updated_at='2025-07-14T11:44:51.123996+00:00')]

In [ ]:
memory.workflows.delete("blub-2", cascade=True)
# memory.workflows.delete(res.workflow_id, cascade=True)

try:
    memory.workflows.get(res.workflow_id)
except Exception as e:
    print(f"Workflow with ID {res.workflow_id} successfuly deleted")

Workflow with ID b4ab782ce1f04e8995e8061bd5102930 successfuly deleted


### WorkflowItems

In [ ]:
from agentmemory.schema.workflows import WorkflowStep, WorkflowStatus

step = WorkflowStep(
    workflow_id="blub-1",
    name="New Workflow Step",
    tool="test_tool",
    arguments={"arg1": "value1", "arg2": "value2"},
    status=WorkflowStatus.RUNNING
)

step_2 = WorkflowStep(
    workflow_id="blub-2",
    name="New Workflow Step 2",
    tool="test_tool_2",
    arguments={"arg1": "value1", "arg2": "value2"},
    status=WorkflowStatus.ERROR
)

step_created = memory.workflow_steps.create(step)
step_created_2 = memory.workflow_steps.create(step_2)
step_created

WorkflowStep(workflow_id='blub-1', name='New Workflow Step', tool='test_tool', arguments={'arg1': 'value1', 'arg2': 'value2'}, status=<WorkflowStatus.RUNNING: 'running'>, _id='6874edd46a0f76356d561c87', step_id='16fd8d4fbfa94748be5a13e1accb7e4f', result=None, logs=[], error=None, data={}, created_at='2025-07-14T11:45:24.829019+00:00', updated_at='2025-07-14T11:45:24.829031+00:00')

In [ ]:
memory.workflow_steps.get("blub-1", step_created.step_id)

WorkflowStep(workflow_id='blub-1', name='New Workflow Step', tool='test_tool', arguments={'arg1': 'value1', 'arg2': 'value2'}, status='running', _id=ObjectId('6874edd46a0f76356d561c87'), step_id='16fd8d4fbfa94748be5a13e1accb7e4f', result=None, logs=[], error=None, data={}, created_at='2025-07-14T11:45:24.829019+00:00', updated_at='2025-07-14T11:45:24.829031+00:00')

In [ ]:
step_created.name = "Updated Workflow Step Name"
step_updated = memory.workflow_steps.update(step_created)
step_updated

WorkflowStep(workflow_id='blub-1', name='Updated Workflow Step Name', tool='test_tool', arguments={'arg1': 'value1', 'arg2': 'value2'}, status=<WorkflowStatus.RUNNING: 'running'>, _id='6874edd46a0f76356d561c87', step_id='16fd8d4fbfa94748be5a13e1accb7e4f', result=None, logs=[], error=None, data={}, created_at='2025-07-14T11:45:24.829019+00:00', updated_at='2025-07-14T11:45:28.273373+00:00')

In [ ]:
[s for s in memory.workflow_steps.list()]

[WorkflowStep(workflow_id='a62c96a5a60544ee97309035de639985', name='New Workflow Step', tool='test_tool', arguments={'arg1': 'value1', 'arg2': 'value2'}, status='running', _id=ObjectId('6874cb447c0672cc31a5d0ba'), step_id='1b7b7f0c50af41dda46b42b622b0ae24', result=None, logs=[], error=None, data={}, created_at='2025-07-14T09:17:56.628638+00:00', updated_at='2025-07-14T09:17:56.628641+00:00'),
 WorkflowStep(workflow_id='a62c96a5a60544ee97309035de639985', name='Updated Workflow Step Name', tool='test_tool', arguments={'arg1': 'value1', 'arg2': 'value2'}, status='running', _id=ObjectId('6874cc397c0672cc31a5d0bb'), step_id='d28eb09cc09344dbb52bbb440f0c5aea', result=None, logs=[], error=None, data={}, created_at='2025-07-14T09:22:01.797660+00:00', updated_at='2025-07-14T09:23:02.639175+00:00'),
 WorkflowStep(workflow_id='a62c96a5a60544ee97309035de639985', name='New Workflow Step 2', tool='test_tool_2', arguments={'arg1': 'value1', 'arg2': 'value2'}, status='error', _id=ObjectId('6874cc397c0

In [ ]:
[s for s in memory.workflow_steps.list_by_workflow_id("blub-2")]

[WorkflowStep(workflow_id='blub-2', name='New Workflow Step 2', tool='test_tool_2', arguments={'arg1': 'value1', 'arg2': 'value2'}, status='error', _id=ObjectId('6874cf00ab96d7c72d806f77'), step_id='f1287266ce1144e9a7c7b4351ba7753f', result=None, logs=[], error=None, data={}, created_at='2025-07-14T09:33:52.388699+00:00', updated_at='2025-07-14T09:33:52.388707+00:00'),
 WorkflowStep(workflow_id='blub-2', name='New Workflow Step 2', tool='test_tool_2', arguments={'arg1': 'value1', 'arg2': 'value2'}, status='error', _id=ObjectId('6874cf1f722ac00e151d6958'), step_id='0f883ed244b14943b73b94731c399bee', result=None, logs=[], error=None, data={}, created_at='2025-07-14T09:34:23.098121+00:00', updated_at='2025-07-14T09:34:23.098130+00:00'),
 WorkflowStep(workflow_id='blub-2', name='New Workflow Step 2', tool='test_tool_2', arguments={'arg1': 'value1', 'arg2': 'value2'}, status='error', _id=ObjectId('6874edd46a0f76356d561c88'), step_id='3b98b32f96a649889f0a4b7d3ceedea1', result=None, logs=[], 

In [ ]:
memory.workflow_steps.delete(step_created.workflow_id, step_created.step_id)

try:
    memory.workflow_steps.get(step_created.workflow_id, step_created.step_id)
except Exception as e:
    print(f"Workflow with ID {step_created.step_id} successfuly deleted")

Workflow with ID 16fd8d4fbfa94748be5a13e1accb7e4f successfuly deleted


## Cache

### General

In [3]:
memory.cache.set("key", "value", ex=100)

True

In [4]:
memory.cache.keys("*")

[b'key']

In [9]:
memory.cache.get("key")

'value'

In [7]:
keys = memory.cache.keys("*")
memory.cache.clear(keys)

### Conversation Caching

In [22]:
keys = memory.cache.keys("*")
memory.cache.clear(keys)

CLEAR KEYS: []


In [30]:
from agentmemory.schema.conversations import Conversation

conversation_count = 5

print("- - - - - - INIT - - - - -")

delete_all_conversations()

print("- - - - - CREATE-BATCH - - - - -")

conversations = [Conversation(f"C{i}") for i in range(0, conversation_count, 1)]
for conversation in conversations:
    memory.conversations.create(conversation)

assert len(memory.conversations.list()) == conversation_count

print("- - - - - CREATE-ONE - - - - -")

conversation_count += 1
conversation_11 = Conversation("Blub-11")
memory.conversations.create(conversation_11)

assert len(memory.conversations.list()) == conversation_count

memory.conversations.get(conversation_11.conversation_id)

print("- - - - - UPDATE - - - - -")

conversation_11.data = {"k": "v"}
memory.conversations.update(conversation_11)

assert memory.conversations.get(conversation_11.conversation_id).data["k"] == "v"
assert memory.conversations.list()[-1].data["k"] == "v"

print("- - - - - DELETE - - - - -")

conversation_count -= 1
memory.conversations.delete(conversation_11.conversation_id, cascade=True)

assert len(memory.conversations.list()) == conversation_count
try:
    res = memory.conversations.get(conversation_11.conversation_id)
    raise Exception("conversation_11 should be None and raise error")
except Exception:
    pass

limit = 3
all_list = memory.conversations.list()
limit_list = memory.conversations.list(limit=limit)
assert len(limit_list) == limit
assert limit_list[-1].title == all_list[-1].title

print("- - - - - CHECK KEYS - - - - -")
print(memory.cache.keys("*"))

- - - - - - INIT - - - - -
CLEAR KEYS: [b'type:LIST;col:conversations;limit:3', b'type:LIST;col:conversations;']
CLEAR KEYS: []
CLEAR KEYS: []
CLEAR KEYS: []
CLEAR KEYS: []
- - - - - CREATE-BATCH - - - - -
CLEAR KEYS: []
CLEAR KEYS: []
CLEAR KEYS: []
CLEAR KEYS: []
CLEAR KEYS: []
- - - - - CREATE-ONE - - - - -
CLEAR KEYS: [b'type:LIST;col:conversations;']
- - - - - UPDATE - - - - -
CLEAR KEYS: [b'type:GET;col:conversations;id:731720018acb4451a4f11f8e562a8002;', b'type:LIST;col:conversations;']
- - - - - DELETE - - - - -
CLEAR KEYS: [b'type:GET;col:conversations;id:731720018acb4451a4f11f8e562a8002;', b'type:LIST;col:conversations;']
FROM CACHE: [{'title': 'C0', '_id': '6878c6e82cdac1413a84b047', 'conversation_id': 'a4f499f31efd4d33b73fa88ea877a330', 'data': '{}', 'created_at': '2025-07-17T09:48:24.212542+00:00', 'updated_at': '2025-07-17T09:48:24.212548+00:00'}, {'title': 'C1', '_id': '6878c6e82cdac1413a84b048', 'conversation_id': '10110816932f4499a7e4506baa55e31a', 'data': '{}', 'creat

In [33]:
print(memory.conversations.list())
print(memory.conversations.list(limit=limit))

FROM CACHE: [{'title': 'C0', '_id': '6878c6e82cdac1413a84b047', 'conversation_id': 'a4f499f31efd4d33b73fa88ea877a330', 'data': '{}', 'created_at': '2025-07-17T09:48:24.212542+00:00', 'updated_at': '2025-07-17T09:48:24.212548+00:00'}, {'title': 'C1', '_id': '6878c6e82cdac1413a84b048', 'conversation_id': '10110816932f4499a7e4506baa55e31a', 'data': '{}', 'created_at': '2025-07-17T09:48:24.212554+00:00', 'updated_at': '2025-07-17T09:48:24.212555+00:00'}, {'title': 'C2', '_id': '6878c6e82cdac1413a84b049', 'conversation_id': 'c3ada413e79643febd70563d896b6561', 'data': '{}', 'created_at': '2025-07-17T09:48:24.212559+00:00', 'updated_at': '2025-07-17T09:48:24.212560+00:00'}, {'title': 'C3', '_id': '6878c6e82cdac1413a84b04a', 'conversation_id': '62e59b451a8f44bc8c18529abb3f3cae', 'data': '{}', 'created_at': '2025-07-17T09:48:24.212563+00:00', 'updated_at': '2025-07-17T09:48:24.212564+00:00'}, {'title': 'C4', '_id': '6878c6e82cdac1413a84b04b', 'conversation_id': 'a732c66349fe4d10bf2d01a9601bb03b

### ConversationItems

In [23]:
keys = memory.cache.keys("*")
memory.cache.clear(keys)

CLEAR KEYS: []


In [ ]:
from agentmemory.schema.conversations import Conversation, ConversationItem

conversation_count = 5

print("- - - - - - INIT - - - - -")

delete_all_conversations()

print("- - - - - CREATE-BATCH - - - - -")
conversation_1 = Conversation("Conversation")
memory.conversations.create(conversation_1)

conversation_items = [
    ConversationItem(
        conversation_id=conversation_1.conversation_id,
        role=f"role{i}",
        content=f"content{i}"
    )
    for i in range(0, conversation_count, 1)
]
for item in conversation_items:
    memory.conversation_items.create(item)

assert len(memory.conversation_items.list()) == conversation_count

print("- - - - - CREATE-ONE - - - - -")

conversation_count += 1
conversation_item_11 = ConversationItem(
    conversation_id=conversation_1.conversation_id,
    role="role11",
    content="content"
)

memory.conversation_items.create(conversation_item_11)
print("LEN:", len(memory.conversation_items.list()))

assert len(memory.conversation_items.list()) == conversation_count

memory.conversation_items.get(conversation_item_11.conversation_id, conversation_item_11.item_id)

print("- - - - - UPDATE - - - - -")

conversation_item_11.data = {"k": "v"}
memory.conversation_items.update(conversation_item_11)

assert memory.conversation_items.get(
    conversation_item_11.conversation_id, conversation_item_11.item_id
    ).data["k"] == "v"
assert memory.conversation_items.list()[-1].data["k"] == "v"

print("- - - - - DELETE - - - - -")

conversation_count -= 1
memory.conversation_items.delete(
    conversation_item_11.conversation_id,
    conversation_item_11.item_id)

assert len(memory.conversation_items.list()) == conversation_count
try:
    res = memory.conversation_items.get(
        conversation_1.conversation_id, conversation_item_11.conversation_id)
    raise Exception("conversation_item_11 should be None and raise error")
except Exception:
    pass

print("- - - - - CHECK LIMIT - - - - -")

limit = 3
all_list = memory.conversation_items.list()
limit_list = memory.conversation_items.list(limit=limit)
assert len(limit_list) == limit
assert limit_list[-1].content == all_list[-1].content

print("- - - - - CHECK KEYS - - - - -")
print(memory.cache.keys("*"))

In [4]:
from agentmemory.schema.conversations import Conversation, ConversationItem

conversation_count = 5

print("- - - - - - INIT - - - - -")

delete_all_conversations()
keys = memory.cache.keys("*")
memory.cache.clear(keys)

print("- - - - - CREATE-BATCH - - - - -")
conversation_1 = Conversation("Conversation-1")
conversation_2 = Conversation("Conversation-2")
memory.conversations.create(conversation_1)
memory.conversations.create(conversation_2)

for i, conv in enumerate([conversation_1, conversation_2]):
    conversation_items = [
        ConversationItem(
            conversation_id=conv.conversation_id,
            role=f"role{i}-{y}",
            content=f"content{i}-{y}"
        )
        for y in range(0, conversation_count, 1)
    ]
    for item in conversation_items:
        memory.conversation_items.create(item)

assert len(memory.conversation_items.list()) == conversation_count * 2

assert len(memory.conversation_items.list_by_conversation_id(conversation_1.conversation_id)) == conversation_count
assert len(memory.conversation_items.list_by_conversation_id(conversation_2.conversation_id)) == conversation_count

assert len(memory.conversation_items.list_until_id_found(conversation_2.conversation_id)) == conversation_count
assert len(memory.conversation_items.list_until_id_found(conversation_2.conversation_id)) == conversation_count

assert len( memory.cache.keys("*")) == 3

# print("- - - - - CREATE-ONE - - - - -")

conversation_item_11 = ConversationItem(
    conversation_id=conversation_1.conversation_id,
    role="role11",
    content="content"
)
memory.conversation_items.create(conversation_item_11)

keys = memory.cache.keys("*")
assert len(keys) == 1
assert conversation_2.conversation_id in keys[0]


display(memory.cache.keys("*"))


memory.conversation_items.list_until_id_found()
# print("LEN:", len(memory.conversation_items.list()))

# assert len(memory.conversation_items.list()) == conversation_count

# memory.conversation_items.get(conversation_item_11.conversation_id, conversation_item_11.item_id)

# print("- - - - - UPDATE - - - - -")

# conversation_item_11.data = {"k": "v"}
# memory.conversation_items.update(conversation_item_11)

# assert memory.conversation_items.get(
#     conversation_item_11.conversation_id, conversation_item_11.item_id
#     ).data["k"] == "v"
# assert memory.conversation_items.list()[-1].data["k"] == "v"

# print("- - - - - DELETE - - - - -")

# conversation_count -= 1
# memory.conversation_items.delete(
#     conversation_item_11.conversation_id,
#     conversation_item_11.item_id)

# assert len(memory.conversation_items.list()) == conversation_count
# try:
#     res = memory.conversation_items.get(
#         conversation_1.conversation_id, conversation_item_11.conversation_id)
#     raise Exception("conversation_item_11 should be None and raise error")
# except Exception:
#     pass

# print("- - - - - CHECK LIMIT - - - - -")

# limit = 3
# all_list = memory.conversation_items.list()
# limit_list = memory.conversation_items.list(limit=limit)
# assert len(limit_list) == limit
# assert limit_list[-1].content == all_list[-1].content

# print("- - - - - CHECK KEYS - - - - -")
# print(memory.cache.keys("*"))

- - - - - - INIT - - - - -
CLEAR KEYS: [b'type:LIST;col:conversations;'] 3
- - - - - CREATE-BATCH - - - - -
CLEAR KEYS: [b'type:LIST_BY_ANCHOR;col:conversation_items;id:06972bf2c5fa405d893c4b16495a7977;', b'type:LIST;col:conversation_items;'] 2


['type:LIST_BY_ANCHOR;col:conversation_items;id:97d0f57120a14ba1a9ebb2c62bc32920;']

In [ ]:
# WICHTIG: anchor logik testen bei CREATE und UPDATE, DELETE
# list_by_conversation_id -> da darf nur nach anchor key gecleart werden; andere müssen bleiben!

### CacheKeys

In [5]:
from agentmemory.connection.shortterm.cache import (
    CacheKey, ClearCacheKey, CacheRetrieveType, ClearCacheTransactionType
)
from agentmemory.connection.longterm.collections import Collection

cache_key = CacheKey(
    rtype=CacheRetrieveType.LIST,
    col=Collection.CONVERSATIONS,
    # query={"data.blub": "rap-1"},
    id=("blub-10", "blub-90")
)

clear_cache_keys = ClearCacheKey(
    id=("blub-10", "blub-90"),
    ttype=ClearCacheTransactionType.DELETE,
    col=Collection.CONVERSATIONS,
    is_first_id_anchor=True
)

print("Cache Key:", cache_key)
print("Clear Key:", clear_cache_keys)

Cache Key: type:LIST;col:conversations;id:blub-10,blub-90
Clear Key: ['type:GET;col:conversations;id:blub-10,blub-90*', 'type:LIST;col:conversations;id:blub-10*']


## CosmosDB Connector

# Psydo Code (tmp)

In [ ]:
from agentmemory import AgentMemory
from agentmemory.persona import Persona
from agentmemory.connection import AgentMemoryConnection
from agentmemory.connection.longterm import MongoDBConnection, CosmosDBConnection
from agentmemory.connection.shortterm import RedisConnection


con = AgentMemoryConnection(
    longterm=MongoDBConnection(
        host="localhost",
        port=27017,
        database="agentmemory",
        username="admin",
        password="password"
    ),
    shortterm=RedisConnection(
        host="localhost",
        port=6379,
        db=0
    )
)

memory = AgentMemory(
    name="typecode-agent",
    con=con
)

personas = [
    Persona()
]

memory.personas.add(personas)

memory.initialize()  # runs schema creations
# conversations
# messages
# workflows
# tools: NICHT in MongoDB, sondern über MCP!

In [ ]:
from agentmemory.examples import create_example_memory

memory = create_example_memory(name="example-memory")

## Work with AgentMemory

### Cache

In [ ]:
memory.cache.keys("*")

In [ ]:
from agentmemory.memory.cache import CacheKey

key = CacheKey(
    (blub, id),
    (blab, id2)
)
# blub:id;blab:id2 ==> oder wie ist best practice?
memory.cache.set("blub:id", "value")

In [ ]:
memory.cache.get("*")

In [ ]:
memory.cache.clear("*")  # or remove, delete, etc.?

### Conversations

In [ ]:
conversation = memory.conversations.get("acfe32d4-e446-4574-84a5-7416e69a9c77")

```json
{
    "_id": ObjectId("acfe32d4-e446-4574-84a5-7416e69a9c77"),
    "conversation_id": "",
    "title": "title",
    "metadata": {...},
    "created_at": "2025-04-24T06:15:50.773544",
    "updated_at": "2025-04-24T06:15:50.773544"
}
```

In [ ]:
conversation = memory.conversations.create(
    name="blub",
    metadata={}
)

In [ ]:
conversation.update({
    "name": "new name",
    "metadata": {
        "key": "value"
    }
})

In [ ]:
memory.conversations.delete("acfe32d4-e446-4574-84a5-7416e69a9c77", cascade=True)


In [ ]:
conversation.delete(cascade=True)

### Conversation Items

In [ ]:
memory.conversation_items.list()

```json
[
    {
        "_id": ObjectId("b90dc6cf-9d3c-4a6d-8f55-5a9e4846b501"),
        "conversation_id: ObjectId("76a9ce2a-4a05-46ad-b364-2106d6dfc0a2"),
        "role": "user",
        "content": "Get me the weather of today",
        "metadata": {...},
        "created_at": "2025-04-24T06:15:50.773544",
        "updated_at": "2025-04-24T06:15:50.773544"
    },
    {
        "_id": ObjectId("7e16d4a5-bcd5-440b-8e70-fc64c4d83629"),
        "conversation_id: ObjectId("76a9ce2a-4a05-46ad-b364-2106d6dfc0a2"),
        "role": "assistant",
        "content": "The weather of today is blub.",
        "metadata": {...},
        "created_at": "2025-04-24T06:15:50.773544",
        "updated_at": "2025-04-24T06:15:50.773544"
    }
]
```

### Personas

In [ ]:
persona = memory.personas.get("acfe32d4-e446-4574-84a5-7416e69a9c77")

persona.to_dict()

```json
{
    "_id": ObjectId("76a9ce2a-4a05-46ad-b364-2106d6dfc0a2"),
    "name": "AI Engineer Cody",
    "role": "Python Software Engineer for FastAPI development",
    "goals": "...",
    "description": "...",
    "embedding": Array(256),
    "created_at": "2025-04-24T06:15:50.773544",
    "updated_at": "2025-04-24T06:15:50.773544"
}

```

In [ ]:
persona.to_dict(fields=["name", "role", "goals", "description"])

In [ ]:
persona.to_text()

In [ ]:
from agentmemory import build_prompt
from agentmemory.instructions import Instructions, StandardInstructions

instructions = Instructions(name="Instructions")
resp_instructions = Instructions(name="Response Instructions")

instructions.add([
    "Do not hallucinate — only use confirmed information."
])
resp_instructions.add([
    StandardInstructions.MARKDOWN_RESPONSE
])

prompt = build_prompt(
    persona=persona,
    instructions=instructions,
    context=""  # context data etc; free to use
)

prompt

### Workflows

In [ ]:
memory.workflows.create()
# https://youtu.be/W2HVdB4Jbjs

In [7]:
import uuid

uuid.uuid4()

UUID('7e16d4a5-bcd5-440b-8e70-fc64c4d83629')